# Mission Control: Geospatial Insights from Falcon 9 Launch Sites

## Project Overview

This notebook complements the main project **Mission Control: Predicting Falcon 9 First Stage Landings with Data Science** by visualizing the geographic dimensions of SpaceX launch activity. While the main analysis explored patterns in payloads, orbits, and landing outcomes, this section shifts focus to **location**—examining how geography and infrastructure may influence launch logistics and recovery operations.

By mapping launch sites and their outcomes, we aim to explore how **geographic context** — proximity to coastlines, distribution across facilities, and success clustering — may reveal additional insights about operational patterns and recovery strategies.

## Objectives

- Visualize all Falcon 9 launch sites on an interactive map
- Overlay markers indicating successful and failed landings
- Investigate spatial patterns and potential influence of location
- Prepare the ground for proximity analysis and spatial modeling 

We use the Folium library to create interactive maps that bring these locations to life.




In [2]:
!pip install folium

In [5]:
# Import essential libraries for geospatial visualization
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
import requests


## Launch Site Mapping

To explore the geographic distribution of SpaceX launch sites, we'll start by mapping the coordinates of each site using **Folium**, a powerful library for interactive geospatial visualizations.

We begin by loading a curated dataset that includes the latitude and longitude of each launch location.

In [6]:
# Download the CSV file directly
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(url)

# Read the CSV content into a DataFrame
with open('spacex_launch_geo.csv', 'wb') as f:
    f.write(response.content)

spacex_df = pd.read_csv('spacex_launch_geo.csv')

We extract the relevant columns (`Launch Site`, `Lat`, and `Long`) to create a reference table for unique launch sites.


In [7]:
# Extract unique launch site coordinates
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Now that we have the coordinates, let’s visualize the sites on a map.

Each site is marked with a colored circle and a labeled marker. This helps uncover spatial patterns and proximity to coastlines or recovery zones, which can impact landing logistics and success rates.

We begin by initializing the map centered around the NASA Johnson Space Center in Houston, TX, then overlay the coordinates of all known Falcon 9 launch locations.


In [10]:

# NASA Johnson Space Center coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize folium map centered on NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for NASA JSC
folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>',
    )
).add_to(site_map)

# Add circles and labels for all launch sites in the launch_sites_df
for _, row in launch_sites_df.iterrows():
    coords = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    folium.Circle(
        coords,
        radius=1000,
        color='#1f618d',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)
    
    folium.map.Marker(
        coords,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#1f618d;"><b>{site_name}</b></div>',
        )
    ).add_to(site_map)

site_map


### Geographic Exploration & Observations

Now that the launch sites are plotted, take a moment to interact with the map—zoom in, zoom out, and examine each location.

Two questions to consider:

- **Are all launch sites in proximity to the Equator?**
- **Are all launch sites near the coast?**

#### Insights:

- **Equatorial Proximity**: While none of the SpaceX launch sites are directly on the Equator, they are all situated in relatively **low-latitude regions** of the U.S., particularly in Florida. This is by design—launching closer to the Equator provides a **velocity boost** due to Earth's rotation, improving fuel efficiency and payload capacity.

- **Coastal Advantage**: All four major launch sites are indeed **very close to coastlines**. This minimizes risk in case of launch failures by allowing rockets to fly over open water, and it supports easier booster recovery—especially for drone ship landings in the Atlantic or Pacific.

These geographical choices highlight SpaceX’s strategic balance between orbital mechanics and operational safety.


### Visualizing Launch Outcomes by Site

To deepen our understanding of launch performance, we overlay individual launch outcomes on the map. Each marker represents a past launch attempt, color-coded by outcome:

- **Green**: Successful landing ('class = 1')
- **Red**: Failed landing ('class = 0')

Since multiple launches occur at the same site coordinates, we use **marker clusters** to prevent overlap and make the map more interactive.

Each marker displays basic launch details when clicked.

This visualization helps reveal spatial trends in launch outcomes and surface location-specific performance insights.


In [11]:
# Create marker color column based on success (class = 1) or failure (class = 0)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through each row in the DataFrame and add markers to the cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}",
        icon=folium.Icon(color=row['marker_color'])
    ).add_to(marker_cluster)

# Add marker cluster to the site map
site_map.add_child(marker_cluster)

# Display the map
site_map


### Geospatial Proximity Analysis

Understanding the **geographic context** of launch sites is critical when evaluating their viability and efficiency. Proximity to infrastructure such as railways, highways, and coastlines can directly impact logistics, safety measures, and recovery operations.

To investigate this, we add an interactive mouse position tracker to our map to allow real-time retrieval of coordinates.

In [12]:
from folium.plugins import MousePosition

# Add mouse position tracker to the map for coordinate lookup
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
   #Calculate the great-circle distance between two points on the Earth's surface using the Haversine formula.
    
    R = 6373.0  # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


### Visualizing Launch Site Proximities

To further analyze geographical influences, we measure and visualize the distance between the launch site and various nearby infrastructures:

- Coastline
- Railway
- Highway
- Nearest city

Each proximity is marked on the map with a distance label, and a colored line is drawn between the site and the location. Different colors are used for each type of proximity to visually distinguish them.


In [18]:
# Function to draw a distance marker and a colored line from launch site to any proximity point
def add_distance_marker(site_lat, site_long, target_lat, target_long, distance, label, color):
    # Add a text label marker for the distance
    marker = folium.Marker(
        location=[target_lat, target_long],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{label}: {distance:.2f} KM</b></div>'
        )
    )

    # Draw a polyline between launch site and target
    line = folium.PolyLine(
        locations=[[site_lat, site_long], [target_lat, target_long]],
        color=color,
        weight=2,
        opacity=0.7
    )

    site_map.add_child(marker)
    site_map.add_child(line)


### Calculating Distance to Nearest Coastline

To better understand the operational logistics of our launch site, we estimate the distance to the nearest coastline. Proximity to the ocean plays a vital role in recovery operations, splashdown zones, and launch safety.

Using the coordinates obtained from the map via `MousePosition`, we calculate the great-circle distance between **CCAFS LC-40** and the closest visible coastline point.

We also visually mark this distance on the map using a `folium.Marker` with a label that displays the computed distance.


In [20]:
# Launch site: CCAFS LC-40
site_lat = 28.562302
site_long = -80.577356

# Closest coastline coordinates
coast_lat = 28.56223
coast_long = -80.56762

# Calculate distance
coast_distance = calculate_distance(site_lat, site_long, coast_lat, coast_long)

# Print the calculated distance
print(f"Distance from CCAFS LC-40 to Coastline: {coast_distance:.2f} KM")

# Add to map with label and line
add_distance_marker(site_lat, site_long, coast_lat, coast_long, coast_distance, "Coastline", "blue")

# Display the updated map
site_map


Distance from CCAFS LC-40 to Coastline: 0.95 KM


### Railway Proximity

To analyze the accessibility of the launch site, we calculate the distance from **CCAFS LC-40** to the nearest railway track. Proximity to railway infrastructure can support cargo transport and heavy equipment delivery to the site.

A green marker shows the distance to the railway from the launch pad.


In [22]:
# Nearest railway coordinates
rail_lat = 28.5564
rail_long = -80.588

# Calculate distance from launch site
rail_distance = calculate_distance(site_lat, site_long, rail_lat, rail_long)
print(f"Distance to nearest railway: {rail_distance:.2f} KM")

# Add marker and polyline to map
add_distance_marker(site_lat, site_long, rail_lat, rail_long, rail_distance, "Railway", "green")

# Display the map
site_map


Distance to nearest railway: 1.23 KM


### Proximity to Highway

The location of a nearby highway was marked and measured from the CCAFS LC-40 launch site. This proximity may be relevant for logistical considerations such as booster transport or recovery operations.


In [23]:
# Highway coordinates
highway_lat = 28.57
highway_long = -80.57

# Calculate distance from launch site to highway
highway_distance = calculate_distance(site_lat, site_long, highway_lat, highway_long)
print(f"Distance to Highway: {highway_distance:.2f} km")

# Add marker and distance label to the map
add_distance_marker(site_lat, site_long, highway_lat, highway_long, highway_distance, "Highway", "green")
site_map

Distance to Highway: 1.12 km


### Proximity to Nearby Cities

Two nearby urban centers—**Titusville** and **Melbourne**—were added to the map to evaluate how geographically isolated or accessible the CCAFS LC-40 launch site is. These distances could influence emergency response planning, staff housing, and logistics.


In [25]:
# Titusville coordinates
titusville_lat = 28.60
titusville_long = -80.78

# Melbourne coordinates
melbourne_lat = 28.05
melbourne_long = -80.62

# Calculate distances
titusville_distance = calculate_distance(site_lat, site_long, titusville_lat, titusville_long)
melbourne_distance = calculate_distance(site_lat, site_long, melbourne_lat, melbourne_long)

# Print distances
print(f"Distance to Titusville: {titusville_distance:.2f} km")
print(f"Distance to Melbourne: {melbourne_distance:.2f} km")

# Add both city markers
add_distance_marker(site_lat, site_long, titusville_lat, titusville_long, titusville_distance, "Titusville", "purple")
add_distance_marker(site_lat, site_long, melbourne_lat, melbourne_long, melbourne_distance, "Melbourne", "orange")
site_map

Distance to Titusville: 20.23 km
Distance to Melbourne: 57.14 km


### Proximity Insights: How Location Influences Launch Logistics

After mapping the distances between the **CCAFS LC-40** launch site and nearby features (coastline, railway, highway, and cities), we can draw the following conclusions:

- **Railway Proximity**:  
  The launch site is **very close (~2 km)** to a railway line, indicating easy access for transporting heavy equipment and rocket components.

- **Highway Access**:  
  A **major highway lies within ~1.2 km** of the site, which supports logistical access for personnel and cargo via ground transport.

- **Coastal Access**:  
  The launch site sits just **~0.9 km from the coastline**, enabling barge transport (like drone ship landings) and making ocean-based landings feasible.

- **Proximity to Cities**:  
  The nearest city, **Titusville**, is **~20.2 km away**, while **Melbourne** is **~57.1 km** distant. This suggests a **buffer zone for safety** and noise concerns while maintaining reasonable access to urban infrastructure and workforce.

### Overall Takeaway

SpaceX’s **strategic launch site placement** balances logistical convenience with safety. Sites are intentionally close to transport infrastructure but **kept at a healthy distance from major population centers**—likely to minimize risk in case of launch failure and reduce disturbance from noise and exhaust plumes.


## Conclusion

This geospatial analysis provided deeper insight into the **location dynamics of SpaceX launch sites**. Using `folium`, we visualized how launch facilities relate to nearby coastlines, highways, railways, and urban areas—factors that can influence both operational logistics and safety.

### Key Learnings:
- **Coastal Placement**: All SpaceX launch sites are positioned near the ocean, aligning with recovery strategies and minimizing risk to populated areas.
- **Infrastructure Access**: Proximity to railways and highways supports efficient transport and deployment.
- **Urban Buffer Zones**: Sites maintain distance from dense population centers, balancing safety with functionality.

To explore these spatial patterns through an interactive lens, see the [SpaceX Dashboard App](/mission-control-falcon9-landing/App/mission_control_dashboard.py), which lets users analyze launch success rates by site, payload mass, and booster version.

---

This notebook complements the [main project analysis](./mission_control_falcon9_analysis.ipynb), which focuses on feature engineering and predictive modeling for Falcon 9 landings.
